In [1]:
# required system modules
from datetime import date
import calendar
import os.path
import platform
import resource
import sys
from tabulate import tabulate
from time import sleep


# required DKC modules
from lib.general import copywrite
from lib.general import error_trap_resource_found
from lib.general import error_trap_resource_not_found
from lib.general import get_availability_domains
from lib.general import get_regions
from lib.general import test_free_mem_1gb
from lib.general import warning_beep
from lib.backups import create_bootvolume_backup
from lib.backups import create_volume_backup
from lib.backups import delete_bootvolume_backup
from lib.backups import delete_volume_backup
from lib.backups import get_compartment_backup_data
from lib.compartments import GetParentCompartments
from lib.compartments import GetChildCompartments
from lib.compute import get_block_vol_attachments
from lib.compute import get_boot_vol_attachments
from lib.compute import GetInstance
from lib.volumes import GetVolumes
from lib.volumes import GetVolumeAttachment

# required OCI modules
from oci.config import from_file
from oci.identity import IdentityClient
from oci.core import BlockstorageClient
from oci.core import ComputeClient
from oci.core.models import CreateBootVolumeBackupDetails
from oci.core.models import CreateVolumeBackupDetails

In [2]:
parent_compartment_name        = "admin_comp"
child_compartment_name         = "oem_comp"
virtual_machine_name           = "EM-OMS1"
region                         = "us-ashburn-1"

In [3]:
# instiate the environment and validate that the specified region exists
config = from_file() # gets ~./.oci/config and reads to the object
identity_client = IdentityClient(config)
regions = get_regions(identity_client)
correct_region = False
for rg in regions:
    if rg.name == region:
        correct_region = True
if not correct_region:
    print("\n\nWARNING! - Region {} does not exist in OCI. Please try again with a correct region.\n\n".format(
        region
    ))
    raise RuntimeWarning("WARNING! INVALID REGION")

config["region"] = region # dictionary object
identity_client = IdentityClient(config) # identity instance
compute_client = ComputeClient(config) # compute instance
storage_client = BlockstorageClient(config) # storage instance primary region


In [4]:
# Get the parent compartment
parent_compartments = GetParentCompartments(parent_compartment_name, config, identity_client)
parent_compartments.populate_compartments()
parent_compartment = parent_compartments.return_parent_compartment()
error_trap_resource_not_found(
    parent_compartment,
    "Parent compartment " + parent_compartment_name + " not found within tenancy " + config["tenancy"]
)

In [5]:
# get the child compartment
child_compartments = GetChildCompartments(
    parent_compartment.id,
    child_compartment_name,
    identity_client)
child_compartments.populate_compartments()
child_compartment = child_compartments.return_child_compartment()
error_trap_resource_not_found(
    child_compartment,
    "Child compartment " + child_compartment_name + " not found within parent compartment " + parent_compartment_name
)

In [6]:
# Get the availability domains for the source VM
availability_domains = get_availability_domains(
    identity_client,
    child_compartment.id)

In [7]:
# Get the VM data but do not pass a VM instance name, we do not need it in this use case
vm_instances = GetInstance(
    compute_client,
    child_compartment.id,
    virtual_machine_name
)
vm_instances.populate_instances()

In [8]:
'''

Fetch all compartment backup data. The underlying function uses the page method in the OCI REST APIs since
the amount of data is copious. Your docker image instance must reserve 1GB RAM for this to work. Begin
by checking this. The function get_compartment_backup_data is complex and required many methods and
functions to run. We ensure this requirement is met by requiring you to pass them to the function.

Explanation of required methods and where you can find them are:

    compute_client                 You should instiate in accordance with KENT codification standards
    get_block_vol_attachments      import from lib.storage
    get_boot_vol_attachments       import from lib.storage
    storage_client                 You should instiate in accordance with KENT codification standards
    child_compartment              You should instiate in accordance with KENT codification standards
    vm_instances                   You should instiate in accordance with KENT codification standards
    
Be sure to read the function backups.get_compartment_backup_data() and get familiar with it before
calling this function. There are comments in the code that explains what it does.

'''
if not test_free_mem_1gb():
    raise RuntimeError("ERROR! INSUFFICENT MEMORY\n\n")
else:
    
    all_compartment_backup_data = get_compartment_backup_data(
        compute_client,
        get_block_vol_attachments,
        get_boot_vol_attachments,
        storage_client,
        child_compartment,
        vm_instances
        )

In [9]:
# set today's backup set name, This will be used to remove old backups and create new ones
current_time_stamp             = date.today()
day_of_month                   = current_time_stamp.day
backup_prefix                  = "_backup_day_of_month_" + str(day_of_month)
bootvolume_backup_sets  = []

volume_backup_sets      = []



for vm in all_compartment_backup_data:
    if vm["vm_name"] == virtual_machine_name:
        
        for boot_volume in vm["boot_volumes"]:
            bootvolume_backup_set_name = {
                "boot_volume_id"    : "",
                "display_name"      : ""
            }
            bootvolume_backup_set_name["boot_volume_id"]   = boot_volume.id
            bootvolume_backup_set_name["display_name"]     = boot_volume.display_name + backup_prefix
            bootvolume_backup_sets.append(bootvolume_backup_set_name)

        for volume in vm["volumes"]:
            volume_backup_set_name = {
                "volume_id"         : "",
                "display_name"      : ""
            }
            volume_backup_set_name["volume_id"]            = volume.id
            volume_backup_set_name["display_name"]         = volume.display_name + backup_prefix
            volume_backup_sets.append(volume_backup_set_name)

In [10]:
bootvolume_backup_sets

[{'boot_volume_id': 'ocid1.bootvolume.oc1.iad.abuwcljt44j5rordpo3bhg7hjse4yuxxtj6n5mvf5bc5h37jgylsxmomhlmq',
  'display_name': 'EM-OMS1 (Boot Volume)_backup_day_of_month_11'}]

In [11]:
volume_backup_sets

[{'volume_id': 'ocid1.volume.oc1.iad.abuwcljtnjauvnd2qcba6w2emzo5qp2p2almvygnxtokt4vwkzmlnpaao2ga',
  'display_name': 'EM-OMS1_datadisk_00_backup_day_of_month_11'},
 {'volume_id': 'ocid1.volume.oc1.iad.abuwcljtdxsos4ppffmo6eh47nmvnmhq3abe4kvdr7np3uzvx4rua537stiq',
  'display_name': 'EM-OMS1_datadisk_01_backup_day_of_month_11'}]

In [12]:
# def delete_bootvolume_backup(
#     storage_client,
#     boot_volume_backup_id
#     ):
#     '''
#     This simple function deletes a boot volume backup. Your code's logic must handle
#     all the pre-reqs.
#     '''
#     delete_boot_volume_backup_response = storage_client.delete_boot_volume_backup(
#         boot_volume_backup_id = boot_volume_backup_id
#     )
    
#     # The returned object does not return anything for the .data method
#     return delete_boot_volume_backup_response.request_id
    
# # end function delete_bootvolume_backup

In [13]:
# def delete_volume_backup(
#     storage_client,
#     volume_backup_id
#     ):
#     '''
#     This simple function deletes a volume backup. Your code's logic must handle
#     all the pre-reqs.
#     '''
#     delete_volume_backup_response = storage_client.delete_volume_backup(
#         volume_backup_id = volume_backup_id
#     )
    
#     # The returned object does not return anything for the .data method
#     return delete_volume_backup_response.request_id

# # end function delete_volume_backup

In [14]:
# First check to see if an old backup snap of the same name exists, and if it does, remove it
for vm in all_compartment_backup_data:
    if vm["vm_name"] == virtual_machine_name:
        
        # first part checks the bootvol backup list
        for bootvbk in vm["boot_volume_backups"]:
            for bootvolume_backup_set_name in bootvolume_backup_sets:
                '''
                The logic necessitates that we abort if the backup snap is in any of the following states.
                This elininates risk of duplicates if the resource is creating and the code called again,
                or if the SNAP is faulty. If the SNAP is faulty, we do not want the code to run. Instead,
                we want the underlying issue to be resolved ASAP. Same logic repeated below.
                '''
                if bootvolume_backup_set_name["display_name"] == bootvbk.display_name and bootvbk.lifecycle_state in [
                "CREATING", "FAULTY", "UNKNOWN_ENUM_VALUE"]:
                    raise RuntimeError("ERROR! BOOT VOLUME BACKUP NOT IN AVAILABLE LIFECYCLE STATE\n\n")
                # do not consider other lifecycle states other than AVAILABLE
                elif bootvolume_backup_set_name["display_name"] == bootvbk.display_name and bootvbk.lifecycle_state == "AVAILABLE":
                    delete_bootvolume_backup(
                        storage_client,
                        bootvbk.id)
        
        # next do the same for block volumes
        for blockvbk in vm["vol_backups"]:
            for volume_backup_set_name in volume_backup_sets:
                if volume_backup_set_name["display_name"] == blockvbk.display_name and blockvbk.lifecycle_state in [
                "CREATING", "FAULTY", "UNKNOWN_ENUM_VALUE"]:
                    raise RuntimeError("ERROR! VOLUME BACKUP NOT IN AVAILABLE LIFECYCLE STATE\n\n")
                elif volume_backup_set_name["display_name"] == blockvbk.display_name and blockvbk.lifecycle_state == "AVAILABLE":
                    delete_volume_backup(
                        storage_client,
                        blockvbk.id)
            
        

In [15]:
# def create_bootvolume_backup(
#     CreateBootVolumeBackupDetails,
#     storage_client,
#     boot_volume_id,
#     display_name,
#     day_of_month
#     ):
#     '''
#     This function creates a boot volume backup. We set the value of backup_type to FULL
#     for 4 days per month. Otherwise we always rumn an incremental backup. Our codification
#     standard requires that we not nest creating the reosurces details within the create
#     operation. This is to make the code more readable even though it is a bit more
#     verbose.
#     '''
#     if day_of_month in [1,8,15,22]:
#         backup_type = "FULL"                # run a full backup 4 times per month
#     else:
#         backup_type = "INCREMENTAL"         # run an incremental backup if not full
#                                             # the first incremental will be full if no full exists for boot volume

#     create_boot_volume_backup_details = CreateBootVolumeBackupDetails(
#         boot_volume_id = boot_volume_id,
#         display_name   = display_name,
#         type           = backup_type
#     )
    
#     create_boot_volume_backup_response = storage_client.create_boot_volume_backup(
#         create_boot_volume_backup_details = create_boot_volume_backup_details
#     )
    
#     # The returned object does contain data within .data, so handle accordingly in your code's logic.
#     return create_boot_volume_backup_response

# # end function create_bootvolume_backup

In [16]:
# def create_volume_backup(
#     CreateVolumeBackupDetails,
#     storage_client,
#     volume_id,
#     display_name,
#     day_of_month
#     ):
#     '''
#     This function creates a volume backup. We set the value of backup_type to FULL
#     for 4 days per month. Otherwise we always rumn an incremental backup. Our codification
#     standard requires that we not nest creating the reosurces details within the create
#     operation. This is to make the code more readable even though it is a bit more
#     verbose.
#     '''
#     if day_of_month in [1,8,15,22]:
#         backup_type = "FULL"                # run a full backup 4 times per month
#     else:
#         backup_type = "INCREMENTAL"         # run an incremental backup if not full
#                                             # the first incremental will be full if no full exists for boot volume
            
#     create_volume_backup_details = CreateVolumeBackupDetails(
#         volume_id = volume_id,
#         display_name = display_name,
#         type = backup_type
#     )
    
#     # The returned object does contain data within .data, so handle accordingly in your code's logic.
#     create_volume_backup_response = storage_client.create_volume_backup(
#         create_volume_backup_details = create_volume_backup_details
#     )
    
#     return create_volume_backup_response

# # end function create_volume_backup

In [17]:
'''
This logic parses down the dictionary object volume_set previously created and passes to
the function create_volume_backup the volume OCID and desired string for display_name.
The OCI REST service will create the SNAP. We choose to not wait for the results but
do receive the full OCI REST response from the function. Note the API does not have logic
do remove old backups. This is why we preceed this logic with code that grooms old
backup SNAPs.
'''

for volume_set in volume_backup_sets:
    create_volume_backup(
    CreateVolumeBackupDetails,
    storage_client,
    volume_set["volume_id"],
    volume_set["display_name"],
    day_of_month)

In [18]:
'''
The same logic as used in the above, except this time we do it for boot volumes.
OCI APIs require a different call for boot volumes versus volumes even though
the logic is identical in both cases.
'''

for boot_volume_set in bootvolume_backup_sets:
    create_bootvolume_backup(
        CreateBootVolumeBackupDetails,
        storage_client,
        boot_volume_set["boot_volume_id"],
        boot_volume_set["display_name"],
        day_of_month
    )